In [1]:
import pandas as pd
import numpy as np

from scipy.optimize import optimize, dual_annealing, shgo, minimize

In [2]:
expert_votes = np.array([4, 5, 2])
crowd_votes = np.array([3, 2, 5, 2, 3, 2, 4, 5, 2, 5, 2, 1, 4, 3, 2, 1])

#### METAHEURISTIC

In [3]:
w_lambda = 0
w_vote = 0

w = [w_lambda, w_vote]
w

[0, 0]

In [4]:
def goal_function(variables, expert_votes = expert_votes, crowd_votes = crowd_votes):
    expert_side = variables[0] * np.mean(np.abs(expert_votes - variables[1]))
    crowd_side = (1 - variables[0]) * np.mean(np.abs(crowd_votes - variables[1]))
    
    return expert_side + crowd_side

In [5]:
results = dual_annealing(goal_function, args=(expert_votes, crowd_votes), 
                         bounds=list(zip([0, 1], [1, 5])), 
                         maxiter=1000, seed=1989)

results

     fun: 1.000000000617628
 message: ['Maximum number of iteration reached']
    nfev: 4223
    nhev: 0
     nit: 1000
    njev: 0
  status: 0
 success: True
       x: array([1., 4.])

In [6]:
results = shgo(goal_function, args=(expert_votes, crowd_votes), 
               bounds=list(zip([0, 1], [1, 5])), 
               n = 5, iters=10)

results

     fun: 1.0000000024835267
    funl: array([1.])
 message: 'Optimization terminated successfully.'
    nfev: 131603
     nit: 10
   nlfev: 18
   nlhev: 0
   nljev: 2
 success: True
       x: array([1.        , 3.99999999])
      xl: array([[1.        , 3.99999999]])

#### NASH

In [7]:
expert_votes = np.array([5, 5, 5])
crowd_votes = np.array([1, 1, 1])

In [8]:
w_lambda_1 = 0.5
w_lambda_2 = 0.5
w_vote = 0

w = [w_lambda_1, w_lambda_2, w_vote]
w

[0.5, 0.5, 0]

In [9]:
def nash_solution(variables, expert_votes = expert_votes, crowd_votes = crowd_votes):
    expert_side = np.log(variables[0]) + np.log(5 - np.mean(np.abs(expert_votes - variables[2])))
    crowd_side = np.log(variables[1]) + np.log(5 - np.mean(np.abs(crowd_votes - variables[2])))
    
    return -1*(expert_side + crowd_side)

def lambda_const(variables):
    return variables[0] + variables[1] - 1

In [10]:
cons = [{'type':'eq', 'fun': lambda_const}]

bnds = ((0.01, 0.99), (0.01, 0.99), (1, 5))

results = minimize(nash_solution, w, 
                   constraints=cons, bounds=bnds, 
                   args=(expert_votes, crowd_votes), method='SLSQP')

results

     fun: -0.8109302161368597
     jac: array([-1.99999810e+00, -2.00000185e+00, -5.91576099e-06])
 message: 'Optimization terminated successfully.'
    nfev: 30
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.50000047, 0.49999953, 2.9999734 ])